In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)  # Displays all the text for each value in a column
pd.set_option('display.float_format', '{:,.2f}'.format)

def format_bigquery_column_names(nftfi):
    # Remove special charachters
    nftfi.columns = [re.sub(r'[^a-zA-Z0-9\s]+', '', column) for column in nftfi.columns]
    # Remove spaces at beginning and end
    nftfi.columns = nftfi.columns.str.strip()
    # Replace spaces with underscore
    nftfi.columns = nftfi.columns.str.replace(' ', '_')
    # Define a function to convert camel-case to kebab-case
    def camel_to_kebab(s):
        # Replace consecutive capital letters with a single lowercase letter
        s = re.sub(r'(?<=[a-z])(?=[A-Z])', '_', s)
        # Convert remaining camel-case string to kebab-case
        s = re.sub(r'(?<!^)(?<!_)(?=[A-Z])(?![A-Z])', '_', s).lower()
        return s
    # Apply the function to all column names
    nftfi.columns = nftfi.columns.map(camel_to_kebab)
    return nftfi



## Load tables

In [2]:
# try: 
#     mq = pd.read_csv('MQ_from_metaquants_telegram.csv')
# except FileNotFoundError:
#     mq = pd.read_csv('analysis/metaquants_analysis/MQ_from_metaquants_telegram.csv')

In [3]:
try: 
    mq = pd.read_csv('data/metaquants_loans.csv')
except FileNotFoundError:
    mq = pd.read_csv('../../data/metaquants_loans.csv')

In [4]:
mq = mq.sort_values(by='block_timestamp', ascending=False)
display(mq.head())
display(mq.protocol.unique())
mq = mq.loc[mq['protocol'] == 'nftfi']
display(mq.protocol.unique())

,transaction_hash,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,protocol,amt_in_usd,roll_over,block_number,p2p_p2pool
17507,0xac6d8958654f8bb61745938f2e46003132f4e3bfcad4d8d88e7f76e25e5e255f,2023-05-18 12:41:47.000000 UTC,36220,0x81007bec2b2c6b211372aa4f2cd340b7713c528b,0x064980edd7d43abee781f49a4e31c06df05ecccb,1.50,1.51,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-05-27 12:41:47.000000 UTC,9.00,0.36,15533,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,nftfi,"2,738.22",False,17286500,p2p
9936,0xba04b7ed490b94b9027db1093e8615fba3aa99e27a74a9623b55530a702130b6,2023-05-18 11:23:47.000000 UTC,36212,0x5e5ccf78a5b6fc0cd2306f0b8723f3986c637535,0xb8a25da593116692444b606be0ed838570b0d7c8,0.47,0.50,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-17 11:23:47.000000 UTC,30.00,0.82,90,0x46c9cfb32627b74f91e0b5ad575c247aec7e7847,nftfi,858.33,False,17286111,p2p
12757,0x0bf87b1d9ecec93f80c864f41899cb206c261c30adad5999318fd42c7ef294f8,2023-05-18 10:16:23.000000 UTC,36207,0xb04a68660362b3c10a6745b14d383a27503c3d50,0x9cd074f0192bb2a477da3271ba03db4fc5cd38c7,0.60,0.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-17 10:16:23.000000 UTC,30.00,0.50,8231,0x5946aeaab44e65eb370ffaa6a7ef2218cff9b47d,nftfi,"1,095.78",False,17285778,p2p
16031,0xf660371bc68bfdcf8810846fa8b418147f4091081fba11c0cfdefa5ddcef8fd4,2023-05-18 09:23:11.000000 UTC,36204,0xc78c3f6cec7d593e0fd931670ca820aac4eee08c,0x5b20dce335d131c6de0cc11b481ace2aade788df,1.30,1.37,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-17 09:23:11.000000 UTC,30.00,0.70,3000118,0x64780ce53f6e966e18a22af13a2f97369580ec11,nftfi,"2,374.36",False,17285519,p2p
17508,0x1bc701828c5f3b8e1e753ae095cd0487e9f431545c816fbe27110a639de53ba7,2023-05-18 08:51:47.000000 UTC,36201,0x81007bec2b2c6b211372aa4f2cd340b7713c528b,0x064980edd7d43abee781f49a4e31c06df05ecccb,1.50,1.51,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-05-27 08:51:47.000000 UTC,9.00,0.36,12015,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,nftfi,"2,740.40",False,17285365,p2p


array(['nftfi'], dtype=object)

array(['nftfi'], dtype=object)

In [5]:
# try: 
#     nftfi = pd.read_csv('../../analytics_bot_langchain/data/nftfi/loans_with_transaction_hash.csv')
# except FileNotFoundError:
#     nftfi = pd.read_csv('analytics_bot_langchain/data/nftfi/loans_with_transaction_hash.csv')
# nftfi = format_bigquery_column_names(nftfi)
# nftfi = nftfi.rename(columns={'loan_id': 'loan_no', 'loan_date': 'date'})
# 
# # Drop nftfi gas price related columns
# for col in nftfi.columns:
#     if 'gas' in col:
#         nftfi = nftfi.drop(columns=[col],axis=1)
#     if 'fee' in col:
#         nftfi = nftfi.drop(columns=[col],axis=1)

In [6]:
try: 
    nftfi = pd.read_csv('../../data/nftfi_loans.csv')
except FileNotFoundError:
    nftfi = pd.read_csv('data/nftfi_loans.csv')
nftfi = format_bigquery_column_names(nftfi)
nftfi = nftfi.rename(columns={'loan_id': 'loan_no', 'loan_date': 'date'})

# Drop nftfi gas price related columns
for col in nftfi.columns:
    if 'gas' in col:
        nftfi = nftfi.drop(columns=[col],axis=1)
    if 'fee' in col:
        nftfi = nftfi.drop(columns=[col],axis=1)

In [7]:
print('MQ')
display(mq.head(2))
print('\n--------------------------------------------------------------------')
print('\n\nNFTfi')
display(nftfi.head(2))

MQ


,transaction_hash,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,protocol,amt_in_usd,roll_over,block_number,p2p_p2pool
17507,0xac6d8958654f8bb61745938f2e46003132f4e3bfcad4d8d88e7f76e25e5e255f,2023-05-18 12:41:47.000000 UTC,36220,0x81007bec2b2c6b211372aa4f2cd340b7713c528b,0x064980edd7d43abee781f49a4e31c06df05ecccb,1.50,1.51,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-05-27 12:41:47.000000 UTC,9.00,0.36,15533,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,nftfi,"2,738.22",False,17286500,p2p
9936,0xba04b7ed490b94b9027db1093e8615fba3aa99e27a74a9623b55530a702130b6,2023-05-18 11:23:47.000000 UTC,36212,0x5e5ccf78a5b6fc0cd2306f0b8723f3986c637535,0xb8a25da593116692444b606be0ed838570b0d7c8,0.47,0.50,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-17 11:23:47.000000 UTC,30.00,0.82,90,0x46c9cfb32627b74f91e0b5ad575c247aec7e7847,nftfi,858.33,False,17286111,p2p



--------------------------------------------------------------------


NFTfi


,loan_no,id,loan_contract,status,borrower,lender,loan_principal_amount,maximum_repayment_amount,loan_duration_d,loan_apr,loan_start_time,date,loan_erc20denomination,block_num,transaction_hash,repaid,liquidated,loan_repaid_time,repaid_amount_paid_to_lender,repaid_transaction_hash,loan_liquidation_date,liquidated_transaction_hash
0,v1.loan.fixed-0,0,v1.loan.fixed,repaid,0xc35A5FEc6BE6957899E15559Be252Db882220b37,0x44DdF3e35Cd2D629d78674FF6BF5b953e2B069fe,30000000000000000,35000000000000000,7,869.05,1589580384,2020-05-15T22:06:24.000Z,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,10073259,0x7c7675a6613ddaded0b187314c2edd3ad3c5428a7a14e92830b74e03fcaf7210,1.00,NaN,2020-05-16T09:25:02.000Z,34750000000000000,0x3e24b9cd4c0208acc25259d4fcbddf75b48a554d68147acd13e656ea025b8c95,NaN,NaN
1,v1.loan.fixed-1,1,v1.loan.fixed,repaid,0xc35A5FEc6BE6957899E15559Be252Db882220b37,0x44DdF3e35Cd2D629d78674FF6BF5b953e2B069fe,3000000000000000,3100000000000000,7,173.81,1589623507,2020-05-16T10:05:07.000Z,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,10076484,0x937a5320404c0c835f2739c84b229ef837a9f0b0ef92cbbf4a232b5b357bfc33,1.00,NaN,2020-05-16T11:50:26.000Z,3095000000000000,0x59d1c26160b7bdde716b380364b506e056d852ac1b360b265109f682a0b89706,NaN,NaN


In [8]:
mq['loan_id'].head(2)

17507    36220
9936     36212
Name: loan_id, dtype: int64

### Clean data

In [9]:
# clean loan IDs to try and compare them
mq['loan_id_cleaned'] = mq['loan_id'].astype(str).str.replace('-','')
nftfi['loan_no_cleaned'] = nftfi['loan_no'].str.split('-').apply(lambda x: x[-1])
nftfi

,loan_no,id,loan_contract,status,borrower,lender,loan_principal_amount,maximum_repayment_amount,loan_duration_d,loan_apr,loan_start_time,date,loan_erc20denomination,block_num,transaction_hash,repaid,liquidated,loan_repaid_time,repaid_amount_paid_to_lender,repaid_transaction_hash,loan_liquidation_date,liquidated_transaction_hash,loan_no_cleaned
0,v1.loan.fixed-0,0,v1.loan.fixed,repaid,0xc35A5FEc6BE6957899E15559Be252Db882220b37,0x44DdF3e35Cd2D629d78674FF6BF5b953e2B069fe,30000000000000000,35000000000000000,7,869.05,1589580384,2020-05-15T22:06:24.000Z,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,10073259,0x7c7675a6613ddaded0b187314c2edd3ad3c5428a7a14e92830b74e03fcaf7210,1.00,NaN,2020-05-16T09:25:02.000Z,34750000000000000,0x3e24b9cd4c0208acc25259d4fcbddf75b48a554d68147acd13e656ea025b8c95,NaN,NaN,0
1,v1.loan.fixed-1,1,v1.loan.fixed,repaid,0xc35A5FEc6BE6957899E15559Be252Db882220b37,0x44DdF3e35Cd2D629d78674FF6BF5b953e2B069fe,3000000000000000,3100000000000000,7,173.81,1589623507,2020-05-16T10:05:07.000Z,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,10076484,0x937a5320404c0c835f2739c84b229ef837a9f0b0ef92cbbf4a232b5b357bfc33,1.00,NaN,2020-05-16T11:50:26.000Z,3095000000000000,0x59d1c26160b7bdde716b380364b506e056d852ac1b360b265109f682a0b89706,NaN,NaN,1
2,v1.loan.fixed-2,2,v1.loan.fixed,repaid,0x44DdF3e35Cd2D629d78674FF6BF5b953e2B069fe,0xc35A5FEc6BE6957899E15559Be252Db882220b37,30000000000000000,30100000000000000,7,17.38,1589628655,2020-05-16T11:30:55.000Z,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,10076867,0xf331e9fb8341f1b644e0edfe599a56402836808b2570578e839261f7c592d2dc,1.00,NaN,2020-05-16T12:20:16.000Z,30095000000000000,0xfa036df32071d8b4f301b34aa2893bc0499db7e79bf12452ad6578820215f683,NaN,NaN,2
3,v1.loan.fixed-3,3,v1.loan.fixed,repaid,0xc35A5FEc6BE6957899E15559Be252Db882220b37,0x44DdF3e35Cd2D629d78674FF6BF5b953e2B069fe,3000000000000000,3010000000000000,7,17.38,1589631935,2020-05-16T12:25:35.000Z,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,10077100,0x330e8f56d4f6eb07392d8c23858eda1cada42c550ed98e4e2bd5bc80ca1be1ff,1.00,NaN,2020-05-16T12:28:51.000Z,3009500000000000,0x8f70de246186ae66da45404c856b9db431b26bfcbe7f100ce405501bbdbff7c2,NaN,NaN,3
4,v1.loan.fixed-4,4,v1.loan.fixed,repaid,0xc35A5FEc6BE6957899E15559Be252Db882220b37,0x44DdF3e35Cd2D629d78674FF6BF5b953e2B069fe,200000000000000000,250000000000000000,7,"1,303.57",1589633522,2020-05-16T12:52:02.000Z,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,10077208,0x9bd7a9da5505b3d42152dd5f971378daeb6a0e74a0ef213b877e27764a06d128,1.00,NaN,2020-05-16T12:56:27.000Z,247500000000000000,0x0eaead5004befd758f4f4a7c04241098eaa9948192e2bb4d10d77358206426e5,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40750,v2-1.loan.fixed-33827,33827,v2-1.loan.fixed,NaN,0x887b86B6B6957F7bbeA88B8CEfD392f39236A88C,0x73A1Bb338613291AC6530870Fe2c5D354BFEC17e,600000000,622191781,30,45.00,1682367647,2023-04-24T20:20:47.000Z,0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48,17118481,0xfd2ae7f526cc0dd5d154e4a24b7d5531201e1b70fab52bd86f0943118bfb5c8f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33827
40751,v2.loan.fixed.collection-33811,33811,v2.loan.fixed.collection,NaN,0xeE8Cd54223A93DD9B3B2C4701cb1Be5f6886C366,0x818ec19eBE023b4774799fdB39F0b20aF39571FE,99000000000000000000,99732329000000000000,30,9.00,1682362607,2023-04-24T18:56:47.000Z,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,17118066,0x1ae59b0a6f064d5877d1cb7daf822fbeb6d861be40f86f6d110e0a2dca318625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33811
40752,v2.loan.fixed.collection-33814,33814,v2.loan.fixed.collection,NaN,0xCffC336E6D019C1aF58257A0b10bf2146a3f42A4,0x1eC627158650fF5dB5d6FCe7C1Ed6e0086D94F5f,52555000000000000000000,52986526945000000000000,30,9.99,1682363507,2023-04-24T19:11:47.000Z,0x6B175474E89094C44Da98b954EedeAC495271d0F,17118141,0xc95639c3fe6ffd6062ab914ce7dcbf693ccbdd4545738f5cfddd6941d74c4447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33814
40753,v2.loan.fixed.collection-33817,33817,v2.loan.fixed.collection,NaN,0xCffC336E6D019C1aF58257A0b10bf2146a3f42A4,0x1eC6

In [10]:
display(mq['loan_id_cleaned'])
display(nftfi['loan_no_cleaned'])

17507    36220
9936     36212
12757    36207
16031    36204
17508    36201
         ...  
11552      347
11553      346
11554      345
11555      344
3048       202
Name: loan_id_cleaned, Length: 22042, dtype: object

0            0
1            1
2            2
3            3
4            4
         ...  
40750    33827
40751    33811
40752    33814
40753    33817
40754    33826
Name: loan_no_cleaned, Length: 40755, dtype: object

In [11]:
mq.shape[0], nftfi.shape[0]

(22042, 40755)

In [12]:
# Check minimum and maximum dates of each table
print(f"min mq date: {mq.block_timestamp.min()}; max mq date: {mq.block_timestamp.max()}")
print(f"min nftfi date: {nftfi.date.min()}; max nftfi date: {nftfi.date.max()}")

min mq date: 2020-10-15 18:15:24.000000 UTC; max mq date: 2023-05-18 12:41:47.000000 UTC
min nftfi date: 2020-05-15T22:06:24.000Z; max nftfi date: 2023-04-24T20:20:47.000Z


In [13]:
mq.loc[mq.loan_id_cleaned.duplicated()==True].sort_values('loan_id_cleaned')

,transaction_hash,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,protocol,amt_in_usd,roll_over,block_number,p2p_p2pool,loan_id_cleaned
17754,0x853126067767f771040544ccf90f6a6e39c6328c442de3ec5e4b88846c205043,2021-07-25 21:41:20.000000 UTC,-1001,0xae00360a54b97723c236093bd272b1b06efc14cd,0x82ba2d6184cb07c46687a89a62c4a9b5c584e87d,30.00,30.49,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2021-08-24 21:41:20.000000 UTC,30.00,0.20,18237,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,nftfi,"64,742.16",False,12897889,p2p,1001
11490,0x40739bfb7332c87bcb4b7037a8d15cb11c68c759a6cc6c9d46c68ab0efd2c980,2021-07-25 22:23:33.000000 UTC,-1002,0xade44991d931ca62e4b56f59a7a8e9160067f48a,0x6954d0255cfed71d55ad8b371b4170b8bf7709dc,0.20,0.20,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2021-10-23 22:23:33.000000 UTC,90.00,0.10,150280,0x50f5474724e0ee42d9a4e711ccfb275809fd6d4a,nftfi,432.45,False,12898074,p2p,1002
17931,0xf6b47521fcee6ae7a0e0489e0254d169e52f52f6942dbbde28961cd08618cc0c,2021-07-26 14:00:25.000000 UTC,-1009,0x538df212def9d27b646b733cb267a69cbe1b77ad,0xf8f327e4c1cbcb0ef014031b84069b9d3579f42d,0.80,0.80,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2021-08-02 14:00:25.000000 UTC,7.00,0.15,6313,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,nftfi,"1,866.91",False,12902241,p2p,1009
3057,0xa6d10395b68628f11032e03dcb7033fd76863e27234a462b3577ef036613cee8,2021-07-26 14:05:44.000000 UTC,-1010,0x44ab88ed750388c1347bbb686dca1ad74119ccdb,0x7dc6f66999983dad3ed2e04ba026e33a693c8457,"15,000.00","16,109.59",0x6b175474e89094c44da98b954eedeac495271d0f,DAI,2021-10-24 14:05:44.000000 UTC,90.00,0.30,52290,0x2a46f2ffd99e19a89476e2f62270e0a35bbf0756,nftfi,"15,000.00",False,12902272,p2p,1010
3056,0x871791b631377ba4e59c9c0aa6a89cb4aa6b28dd092b237a677e634c06512850,2021-07-26 14:08:27.000000 UTC,-1011,0x44ab88ed750388c1347bbb686dca1ad74119ccdb,0x7dc6f66999983dad3ed2e04ba026e33a693c8457,"15,000.00","16,109.59",0x6b175474e89094c44da98b954eedeac495271d0f,DAI,2021-10-24 14:08:27.000000 UTC,90.00,0.30,52399,0x2a46f2ffd99e19a89476e2f62270e0a35bbf0756,nftfi,"15,000.00",False,12902285,p2p,1011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,0x92b50198bde5ec37ff57705d7c67558047df6fc5aef43069c2268f7cebf0d890,2021-07-13 14:53:57.000000 UTC,-869,0xb3ffd0f321abde1740f6e13be273348baad24291,0x0a26e8a4305ad5d9e6844e9d2aad05262b1285e6,1.00,1.08,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2021-10-11 14:53:57.000000 UTC,90.00,0.32,9060,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,nftfi,"1,978.41",False,12819438,p2p,869
339,0x9661633f37ff9db35787db96ffae194f407b2c8abb0a9fba7c1b0bcf02d8d235,2021-07-13 17:29:05.000000 UTC,-870,0x886478d3cf9581b624cb35b5446693fc8a58b787,0x82ba2d6184cb07c46687a89a62c4a9b5c584e87d,1.50,1.52,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2021-08-12 17:29:05.000000 UTC,30.00,0.20,6223,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,nftfi,"2,985.73",False,12820129,p2p,870
721,0x5e951749a79420288a6c04faac8fc8d1c1a18546c3493572f32f8d31f0334917,2021-07-17 15:18:13.000000 UTC,-889,0x3a0ec8481710c8d6c7ef442036a69c4bcc7fdf59,0x2e1aea690847a5cd9cb697f11a40eac956492039,0.00,0.00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2021-10-15 15:18:13.000000 UTC,90.00,0.00,520532,0x06012c8cf97bead5deae237070f9587f8e7a266d,nftfi,7.53,False,12845002,p2p,889
337,0x4e64843d5ec317308038b53c3ed8943c513770d9cb007df6a1693eb8d6fca68d,2021-07-22 15:59:21.000000 UTC,-949,0x8f903cfc0af3c2ec0d872c57538af5e071544a57,0x82ba2d6184cb07c46687a89a62c4a9b5c584e87d,6.00,6.10,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2021-08-21 15:59:21.000000 UTC,30.00,0.20,1026,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,nftfi,"11,994.95",False,12877162,p2p,949


In [14]:
nftfi.head(2)

,loan_no,id,loan_contract,status,borrower,lender,loan_principal_amount,maximum_repayment_amount,loan_duration_d,loan_apr,loan_start_time,date,loan_erc20denomination,block_num,transaction_hash,repaid,liquidated,loan_repaid_time,repaid_amount_paid_to_lender,repaid_transaction_hash,loan_liquidation_date,liquidated_transaction_hash,loan_no_cleaned
0,v1.loan.fixed-0,0,v1.loan.fixed,repaid,0xc35A5FEc6BE6957899E15559Be252Db882220b37,0x44DdF3e35Cd2D629d78674FF6BF5b953e2B069fe,30000000000000000,35000000000000000,7,869.05,1589580384,2020-05-15T22:06:24.000Z,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,10073259,0x7c7675a6613ddaded0b187314c2edd3ad3c5428a7a14e92830b74e03fcaf7210,1.00,NaN,2020-05-16T09:25:02.000Z,34750000000000000,0x3e24b9cd4c0208acc25259d4fcbddf75b48a554d68147acd13e656ea025b8c95,NaN,NaN,0
1,v1.loan.fixed-1,1,v1.loan.fixed,repaid,0xc35A5FEc6BE6957899E15559Be252Db882220b37,0x44DdF3e35Cd2D629d78674FF6BF5b953e2B069fe,3000000000000000,3100000000000000,7,173.81,1589623507,2020-05-16T10:05:07.000Z,0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2,10076484,0x937a5320404c0c835f2739c84b229ef837a9f0b0ef92cbbf4a232b5b357bfc33,1.00,NaN,2020-05-16T11:50:26.000Z,3095000000000000,0x59d1c26160b7bdde716b380364b506e056d852ac1b360b265109f682a0b89706,NaN,NaN,1


In [15]:
print('MQ:\n')
print("Are loan_id unique? Once grouped by unique, the counts are: ")
mq_loan_id_count = pd.DataFrame(mq.groupby('loan_id').agg('count')['transaction_hash'])
mq_loan_id_count = mq_loan_id_count.reset_index()

display(mq_loan_id_count['transaction_hash'].unique())
display(mq_loan_id_count.loc[mq_loan_id_count['transaction_hash']>1])
duplicated_mq_loans = pd.merge(left=mq, right=mq_loan_id_count.loc[mq_loan_id_count['transaction_hash']>1], left_on='loan_id', right_on='loan_id')
display(duplicated_mq_loans)
display(duplicated_mq_loans.amt_in_usd.sum())
print("Are loan_id_cleaned unique? Once grouped by loan_id_cleaned, the counts are: ")
display(mq.groupby('loan_id_cleaned').agg('count')['transaction_hash'].unique())


MQ:

Are loan_id unique? Once grouped by unique, the counts are: 


array([1, 2])

,loan_id,transaction_hash
177,-6545,2
537,-5752,2
538,-5751,2
14879,21523,2
16152,24224,2
19041,30521,2
19042,30522,2
19043,30523,2


,transaction_hash_x,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,protocol,amt_in_usd,roll_over,block_number,p2p_p2pool,loan_id_cleaned,transaction_hash_y
0,0x5d215283e0326ebddc63178230b83a1b00612e9fe30de12c4e52aaea4b2c1879,2023-03-26 16:44:11.000000 UTC,30523,0xbec69dfce4c1fa8b7843fee1ca85788d84a86b06,0x1797b4235473fbe0e7e44322f01c1b5618ebda41,1.00,1.02,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-25 16:44:11.000000 UTC,30.00,0.29,14093,0x942bc2d3e7a589fe5bd4a5c6ef9727dfd82f5c8a,nftfi,"1,764.35",False,16912867,p2p,30523,2
1,0x5d215283e0326ebddc63178230b83a1b00612e9fe30de12c4e52aaea4b2c1879,2023-03-26 16:44:11.000000 UTC,30523,0xbec69dfce4c1fa8b7843fee1ca85788d84a86b06,0x1797b4235473fbe0e7e44322f01c1b5618ebda41,1.00,1.02,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-25 16:44:11.000000 UTC,30.00,0.29,14093,0x942bc2d3e7a589fe5bd4a5c6ef9727dfd82f5c8a,nftfi,"1,764.35",False,16912867,p2p,30523,2
2,0x205c9f50b22b5a4751b14e09efb1eacc757ede76ca421a6b500a0962155ed4e4,2023-03-26 16:43:47.000000 UTC,30522,0x48cf6acbcba1c8fe470827c6739a80c360ccc326,0x0b8428004ab7fea256dbc6e7c6412381cb173e8a,0.60,0.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-09 16:43:47.000000 UTC,14.00,0.79,365,0x64a1c0937728d8d2fa8cd81ef61a9c860b7362db,nftfi,"1,058.61",False,16912865,p2p,30522,2
3,0x205c9f50b22b5a4751b14e09efb1eacc757ede76ca421a6b500a0962155ed4e4,2023-03-26 16:43:47.000000 UTC,30522,0x48cf6acbcba1c8fe470827c6739a80c360ccc326,0x0b8428004ab7fea256dbc6e7c6412381cb173e8a,0.60,0.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-09 16:43:47.000000 UTC,14.00,0.79,365,0x64a1c0937728d8d2fa8cd81ef61a9c860b7362db,nftfi,"1,058.61",False,16912865,p2p,30522,2
4,0xd5f969b32332020a8e7129e4aa2f20a5bff446b31e0054469aa47013268704d7,2023-03-26 16:43:11.000000 UTC,30521,0x48cf6acbcba1c8fe470827c6739a80c360ccc326,0x0b8428004ab7fea256dbc6e7c6412381cb173e8a,0.60,0.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-09 16:43:11.000000 UTC,14.00,0.79,323,0x64a1c0937728d8d2fa8cd81ef61a9c860b7362db,nftfi,"1,058.61",False,16912862,p2p,30521,2
5,0xd5f969b32332020a8e7129e4aa2f20a5bff446b31e0054469aa47013268704d7,2023-03-26 16:43:11.000000 UTC,30521,0x48cf6acbcba1c8fe470827c6739a80c360ccc326,0x0b8428004ab7fea256dbc6e7c6412381cb173e8a,0.60,0.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-09 16:43:11.000000 UTC,14.00,0.79,323,0x64a1c0937728d8d2fa8cd81ef61a9c860b7362db,nftfi,"1,058.61",False,16912862,p2p,30521,2
6,0xfb83f7b98831eec2595e1af36e064057cfac3018b15513a6cb8dd7017c6c4df8,2023-02-06 20:30:11.000000 UTC,24224,0x910d44c699c24cddd03ce8e7e8cc40835464fb89,0x35fa3c19cf39674353025234ed001bcbb36af35c,0.31,0.32,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-03-08 20:30:11.000000 UTC,30.00,0.55,32395,0x942bc2d3e7a589fe5bd4a5c6ef9727dfd82f5c8a,nftfi,511.04,False,16572143,p2p,24224,2
7,0xfb83f7b98831eec2595e1af36e064057cfac3018b15513a6cb8dd7017c6c4df8,2023-02-06 20:30:11.000000 UTC,24224,0x910d44c699c24cddd03ce8e7e8cc40835464fb89,0x35fa3c19cf39674353025234ed001bcbb36af35c,0.31,0.32,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-03-08 20:30:11.000000 UTC,30.00,0.55,32395,0x942bc2d3e7a589fe5bd4a5c6ef9727dfd82f5c8a,nftfi,511.04,False,16572143,p2p,24224,2
8,0xbbd45783e2cd294f86ec604831eeeca7be8bd844818ef7ea52a3df7eb5a01c94,2023-01-18 06:03:35.000000 UTC,21523,0x45d1a3acee4b57f5bb078a14d993364f80e86657,0xebd0db1bfaa50e7abe52412644e968166f349755,0.90,0.92,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-02-01 06:03:35.000000 UTC,14.00,0.69,1662,0x394e3d3044fc89fcdd966d3cb35ac0b32b0cda91,nftfi,"1,428.82",False,16431774,p2p,21523,2
9,0xbbd45783e2cd294f86ec604831eeeca7be8bd844818ef7ea52a3df7eb5a01c94,2023-01-18 06:03:35.000000 UTC,21523,0x45d1a3acee4b57f5bb078a14d993364f80e86657,0xebd0db1bfaa50e7abe52412644e968166f349755,0.90,0.92,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-02-01 06:03:35.000000 U

28611.210136

Are loan_id_cleaned unique? Once grouped by loan_id_cleaned, the counts are: 


array([1, 2, 3])

In [16]:
print(f"duplicate lines in MQ dataset {mq.shape[0] - mq.drop_duplicates().shape[0]}")
print(f"duplicate lines in nftfi dataset {nftfi.shape[0] - nftfi.drop_duplicates().shape[0]}")

duplicate lines in MQ dataset 8
duplicate lines in nftfi dataset 0


## Clean raw NFTfi dataset

In [17]:
# nftfi.dtypes

In [18]:
nftfi['date'] = pd.to_datetime(nftfi['date'], format="%Y-%m-%d %H:%M:%S%z")
nftfi['loan_start_time'] = pd.to_datetime(nftfi['loan_start_time'], format="%Y-%m-%d %H:%M:%S%z")
nftfi['loan_repaid_time'] = pd.to_datetime(nftfi['loan_repaid_time'], format="%Y-%m-%d %H:%M:%S%z")

# Remove invalid values
nftfi.replace(r"#DIV/0!", np.nan, regex=True, inplace=True)
nftfi.replace(r"#N/A", "", regex=True, inplace=True)

# Divide loanPrincipalAmount and maximumRepaymentAmount by ETH <> WEI i.e. 1^18
columns_to_clean = ['loan_principal_amount', 'maximum_repayment_amount', 'maximum_repayment_amount']
for col in columns_to_clean:
    nftfi[col] = nftfi[col].astype(np.float64)
    nftfi[col] /= 10 ** 18
# nftfi['no_of_days'] = nftfi['no_of_days'].astype(np.float64)
nftfi = nftfi.rename(columns={'loan_apr': 'apr'})
nftfi['apr'] = nftfi['apr'].astype(np.float64)

# Drop last column as it is unnamed
nftfi = nftfi.drop('', axis=1, errors='ignore')

# set BOOL columns to bool type
cols = ['repaid', 'liquidated']
nftfi['repaid'] = nftfi['repaid'].fillna(False)
nftfi['repaid'] = nftfi['repaid'].replace('', False)
for col in cols:
    # nftfi[col] = nftfi[col].astype('boolean')
    nftfi[col] = nftfi[col].map({'True': True, 'False': False})
    nftfi[col] = nftfi[col].astype(bool)
nftfi['repaid'] = nftfi['repaid'].astype(bool)

## Enrich datasets
### Enrich NFTfi dataset with ETH price and compute usd_value of loan for weth-denominated loans

In [19]:
# Enrich dataset with ETHUSD rate i.e. ETHPrice, then fill USDValue for USDValue from loanERC20Denomination == weth_address
weth_address = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'.lower()
price_df = pd.read_csv('../../analytics_bot_langchain/data/nftfi/eth_usd_input_data.csv')
price_df = price_df.rename(columns={'datetime': 'date'})
price_df['date'] = pd.to_datetime(price_df['date'], format='%Y-%m-%d %H:%M:%S%z')
price_df = price_df.drop(columns=['open', 'low', 'volume'])

nftfi['date'] = pd.to_datetime(nftfi['date'])
nftfi['loan_principal_amount'] = nftfi['loan_principal_amount'].astype(float)

nftfi = nftfi.sort_values(by='date')
price_df = price_df.sort_values(by='date')

nftfi = pd.merge_asof(nftfi, price_df, on='date', direction='backward')
nftfi['eth_price'] = nftfi['close']
nftfi = nftfi.drop(columns=['close'])
nftfi['loan_erc20denomination'] = nftfi['loan_erc20denomination'].astype(str).str.lower().str.strip()
nftfi.loc[nftfi['loan_erc20denomination'] == weth_address, 'usd_value'] = nftfi.loc[nftfi['loan_erc20denomination'] == weth_address]['loan_principal_amount'] * nftfi.loc[nftfi['loan_erc20denomination'] == weth_address]['eth_price']

# now map missing usd_value, e.g. for stablecoin-denominated loans
usdc_address = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'  # https://etherscan.io/token/0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
usdt_address = '0xdAC17F958D2ee523a2206206994597C13D831ec7'  # https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7
dai_address = '0x6B175474E89094C44Da98b954EedeAC495271d0F'  # https://etherscan.io/token/0x6b175474e89094c44da98b954eedeac495271d0f
stablecoin_addresses = [usdc_address, usdt_address, dai_address]
for stablecoin_address in stablecoin_addresses:
    stablecoin_address = stablecoin_address.lower()
    nftfi.loc[nftfi['loan_erc20denomination'] == stablecoin_address, 'usd_value'] = nftfi.loc[nftfi['loan_erc20denomination'] == stablecoin_address]['loan_principal_amount']



In [20]:
print('\n\nNFTfi: \n')
print("Are loan_no unique? Once grouped by unique, the counts are: ")
nftfi_loan_no_count = pd.DataFrame(nftfi.groupby('loan_no').agg('count')['borrower'])
duplicated_nftfi_loans = pd.merge(left=nftfi, right=nftfi_loan_no_count.loc[nftfi_loan_no_count['borrower']>1], left_on='loan_no', right_on='loan_no')
display(duplicated_nftfi_loans)
display(duplicated_nftfi_loans.usd_value.sum())

display(nftfi_loan_no_count['borrower'].unique())
display(nftfi_loan_no_count.loc[nftfi_loan_no_count['borrower']>1])

print("Are loan_no_cleaned unique? Once grouped by loan_id_cleaned, the counts are: ")
display(nftfi.groupby('loan_no_cleaned').agg('count')['borrower'].unique())



NFTfi: 

Are loan_no unique? Once grouped by unique, the counts are: 


,loan_no,id,loan_contract,status,borrower_x,lender,loan_principal_amount,maximum_repayment_amount,loan_duration_d,apr,loan_start_time,date,loan_erc20denomination,block_num,transaction_hash,repaid,liquidated,loan_repaid_time,repaid_amount_paid_to_lender,repaid_transaction_hash,loan_liquidation_date,liquidated_transaction_hash,loan_no_cleaned,high,eth_price,usd_value,borrower_y


0.0

array([1])

,borrower
loan_no,


Are loan_no_cleaned unique? Once grouped by loan_id_cleaned, the counts are: 


array([1, 2])

### Enrich nftfi table with ERC20 denominations

In [21]:
# Check if it can be due to missing token
usdc_address = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'  # https://etherscan.io/token/0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
usdt_address = '0xdAC17F958D2ee523a2206206994597C13D831ec7'  # https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7
dai_address = '0x6B175474E89094C44Da98b954EedeAC495271d0F'  # https://etherscan.io/token/0x6b175474e89094c44da98b954eedeac495271d0f
weth_address = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'

stablecoin_addresses = [usdc_address, usdt_address, dai_address, weth_address]

stablecoin_names = ['usdc', 'usdt', 'dai', 'weth']
for stablecoin_address, stablecoin_name in zip(stablecoin_addresses, stablecoin_names):
    stablecoin_address = stablecoin_address.lower()
    nftfi.loc[nftfi['loan_erc20denomination'] == stablecoin_address, 'loan_erc20denomination_name'] = stablecoin_name

In [22]:
# Confirm that the above addresses matches the one available in MQ dataset:
mq.erc20_address.unique()

array(['0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
       '0x6b175474e89094c44da98b954eedeac495271d0f',
       '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'], dtype=object)

#### Enrich NFTfi table with loan types

In [23]:
nftfi['loan_type'] = nftfi.loan_no.astype(str).str.split('-').apply(lambda x: x[0]+'-'+x[1] if len(x) >2 else x[0])
nftfi['loan_type'].unique()

array(['v1.loan.fixed', 'v2.loan.fixed', 'v2-1.loan.fixed',
       'v2.loan.fixed.collection'], dtype=object)

In [24]:
mq

,transaction_hash,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,protocol,amt_in_usd,roll_over,block_number,p2p_p2pool,loan_id_cleaned
17507,0xac6d8958654f8bb61745938f2e46003132f4e3bfcad4d8d88e7f76e25e5e255f,2023-05-18 12:41:47.000000 UTC,36220,0x81007bec2b2c6b211372aa4f2cd340b7713c528b,0x064980edd7d43abee781f49a4e31c06df05ecccb,1.50,1.51,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-05-27 12:41:47.000000 UTC,9.00,0.36,15533,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,nftfi,"2,738.22",False,17286500,p2p,36220
9936,0xba04b7ed490b94b9027db1093e8615fba3aa99e27a74a9623b55530a702130b6,2023-05-18 11:23:47.000000 UTC,36212,0x5e5ccf78a5b6fc0cd2306f0b8723f3986c637535,0xb8a25da593116692444b606be0ed838570b0d7c8,0.47,0.50,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-17 11:23:47.000000 UTC,30.00,0.82,90,0x46c9cfb32627b74f91e0b5ad575c247aec7e7847,nftfi,858.33,False,17286111,p2p,36212
12757,0x0bf87b1d9ecec93f80c864f41899cb206c261c30adad5999318fd42c7ef294f8,2023-05-18 10:16:23.000000 UTC,36207,0xb04a68660362b3c10a6745b14d383a27503c3d50,0x9cd074f0192bb2a477da3271ba03db4fc5cd38c7,0.60,0.62,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-17 10:16:23.000000 UTC,30.00,0.50,8231,0x5946aeaab44e65eb370ffaa6a7ef2218cff9b47d,nftfi,"1,095.78",False,17285778,p2p,36207
16031,0xf660371bc68bfdcf8810846fa8b418147f4091081fba11c0cfdefa5ddcef8fd4,2023-05-18 09:23:11.000000 UTC,36204,0xc78c3f6cec7d593e0fd931670ca820aac4eee08c,0x5b20dce335d131c6de0cc11b481ace2aade788df,1.30,1.37,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-17 09:23:11.000000 UTC,30.00,0.70,3000118,0x64780ce53f6e966e18a22af13a2f97369580ec11,nftfi,"2,374.36",False,17285519,p2p,36204
17508,0x1bc701828c5f3b8e1e753ae095cd0487e9f431545c816fbe27110a639de53ba7,2023-05-18 08:51:47.000000 UTC,36201,0x81007bec2b2c6b211372aa4f2cd340b7713c528b,0x064980edd7d43abee781f49a4e31c06df05ecccb,1.50,1.51,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-05-27 08:51:47.000000 UTC,9.00,0.36,12015,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,nftfi,"2,740.40",False,17285365,p2p,36201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11552,0xb28e7b0a6a5bca71f2be910557e7ae4913f08cc6f22b2b51668f4bff2c959e5a,2020-12-10 20:19:28.000000 UTC,-347,0x115b72c42fa4d48777a26f612a26bdecf3d40b2d,0xb167dcc71dd20971ef9cd607cffe7badb6f9cbf5,0.16,0.20,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2021-03-10 20:19:28.000000 UTC,90.00,1.01,92499,0x50f5474724e0ee42d9a4e711ccfb275809fd6d4a,nftfi,90.43,False,11427297,p2p,347
11553,0x0235944275f7263e03df13d42d9d919609315d0fec663ad862fe979ee26d6805,2020-12-10 20:18:57.000000 UTC,-346,0x115b72c42fa4d48777a26f612a26bdecf3d40b2d,0xb167dcc71dd20971ef9cd607cffe7badb6f9cbf5,0.16,0.20,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2021-03-10 20:18:57.000000 UTC,90.00,1.01,110828,0x50f5474724e0ee42d9a4e711ccfb275809fd6d4a,nftfi,90.43,False,11427292,p2p,346
11554,0x3133cfa3ff926e372f78f6e4147a27556d771d2252e8bb221570041a61c6309d,2020-12-10 20:13:25.000000 UTC,-345,0x115b72c42fa4d48777a26f612a26bdecf3d40b2d,0xb167dcc71dd20971ef9cd607cffe7badb6f9cbf5,0.16,0.20,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2021-03-10 20:13:25.000000 UTC,90.00,1.01,92500,0x50f5474724e0ee42d9a4e711ccfb275809fd6d4a,nftfi,90.43,False,11427273,p2p,345
11555,0x33dfe6a14942db0be48a267feae724060554cbec675fa11877d742f9fdc219dc,2020-12-10 19:45:40.000000 UTC,-344,0x115b72c42fa4d48777a26f612a26bdecf3d40b2d,0xb167dcc71dd20971ef9cd607cffe7badb6f9cbf5,0.18,0.19,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2021-01-09 19:45:40.000000 UTC,30.00,0.68,110360,0x50f5474724e0ee42d9a4e711ccfb275809fd6d4a,nftfi,100.79,False,11427141,p2p,344


In [25]:
# enrich dataset with collection name, blockchain
try:
    collection_names_df = pd.read_csv('analytics_bot_langchain/data/nft_address_name_symbol/nft_address_name_symbol.csv')
except FileNotFoundError:
    collection_names_df = pd.read_csv('../../analytics_bot_langchain/data/nft_address_name_symbol/nft_address_name_symbol.csv')
collection_names_df = collection_names_df.rename(columns={'contract_address': 'collection_address', 'name': 'collection_name'})
collection_names_df['collection_address'] = collection_names_df['collection_address'].astype(str).str.lower().str.strip()
collection_names_df = collection_names_df.drop(columns=['symbol', 'standard', 'blockchain'])
display(collection_names_df.head(2))
mq['collection_address'] = mq['collection_address'].astype(str).str.lower().str.strip()

mq_existing_cols = mq.columns
mq = pd.merge(left=mq, right=collection_names_df, on='collection_address', how='left')
new_columns_order = ['collection_name'] + list(mq_existing_cols)
mq = mq[new_columns_order]

,collection_address,collection_name
0,0x3b37270c332b5c6cf6aac0103d9d896f5dcafb1d,Riia the Girls
1,0x668d40fb53871aa139ae306bca88f00bb8c475fb,Simpin Santa


## Check USD value discrepancies for matching loans on transaction hash

In [26]:
display(nftfi.head(2))
display(mq.head(2))

nftfi.columns = [f'nftfi_{col}' for col in nftfi.columns]
mq.columns = [f'mq_{col}' for col in mq.columns]
full_inner = pd.merge(nftfi, mq, how='inner', left_on='nftfi_transaction_hash', right_on='mq_transaction_hash', indicator=True)


,loan_no,id,loan_contract,status,borrower,lender,loan_principal_amount,maximum_repayment_amount,loan_duration_d,apr,loan_start_time,date,loan_erc20denomination,block_num,transaction_hash,repaid,liquidated,loan_repaid_time,repaid_amount_paid_to_lender,repaid_transaction_hash,loan_liquidation_date,liquidated_transaction_hash,loan_no_cleaned,high,eth_price,usd_value,loan_erc20denomination_name,loan_type
0,v1.loan.fixed-0,0,v1.loan.fixed,repaid,0xc35A5FEc6BE6957899E15559Be252Db882220b37,0x44DdF3e35Cd2D629d78674FF6BF5b953e2B069fe,0.03,0.00,7,869.05,1970-01-01 00:00:01.589580384,2020-05-15 22:06:24+00:00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,10073259,0x7c7675a6613ddaded0b187314c2edd3ad3c5428a7a14e92830b74e03fcaf7210,True,True,2020-05-16 09:25:02+00:00,34750000000000000,0x3e24b9cd4c0208acc25259d4fcbddf75b48a554d68147acd13e656ea025b8c95,NaN,NaN,0,NaN,NaN,NaN,weth,v1.loan.fixed
1,v1.loan.fixed-1,1,v1.loan.fixed,repaid,0xc35A5FEc6BE6957899E15559Be252Db882220b37,0x44DdF3e35Cd2D629d78674FF6BF5b953e2B069fe,0.00,0.00,7,173.81,1970-01-01 00:00:01.589623507,2020-05-16 10:05:07+00:00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,10076484,0x937a5320404c0c835f2739c84b229ef837a9f0b0ef92cbbf4a232b5b357bfc33,True,True,2020-05-16 11:50:26+00:00,3095000000000000,0x59d1c26160b7bdde716b380364b506e056d852ac1b360b265109f682a0b89706,NaN,NaN,1,NaN,NaN,NaN,weth,v1.loan.fixed


,collection_name,transaction_hash,block_timestamp,loan_id,to_address,from_address,principal_amount,repayment_amount,erc20_address,erc20_name,due_date,duration_in_days,apr,token_id,collection_address,protocol,amt_in_usd,roll_over,block_number,p2p_p2pool,loan_id_cleaned
0,Meebits,0xac6d8958654f8bb61745938f2e46003132f4e3bfcad4d8d88e7f76e25e5e255f,2023-05-18 12:41:47.000000 UTC,36220,0x81007bec2b2c6b211372aa4f2cd340b7713c528b,0x064980edd7d43abee781f49a4e31c06df05ecccb,1.50,1.51,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-05-27 12:41:47.000000 UTC,9.00,0.36,15533,0x7bd29408f11d2bfc23c34f18275bbf23bb716bc7,nftfi,"2,738.22",False,17286500,p2p,36220
1,NFTfi Locked Bundle,0xba04b7ed490b94b9027db1093e8615fba3aa99e27a74a9623b55530a702130b6,2023-05-18 11:23:47.000000 UTC,36212,0x5e5ccf78a5b6fc0cd2306f0b8723f3986c637535,0xb8a25da593116692444b606be0ed838570b0d7c8,0.47,0.50,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-06-17 11:23:47.000000 UTC,30.00,0.82,90,0x46c9cfb32627b74f91e0b5ad575c247aec7e7847,nftfi,858.33,False,17286111,p2p,36212


In [27]:
usd_diff = full_inner[['mq_amt_in_usd','nftfi_usd_value', 'nftfi_transaction_hash']]# .dropna()
display(usd_diff.head(2))
display(usd_diff.tail(2))
usd_diff['usd_diff_nftfi_minus_mq'] = usd_diff['nftfi_usd_value'] - usd_diff['mq_amt_in_usd']
display(usd_diff.tail(2))

,mq_amt_in_usd,nftfi_usd_value,nftfi_transaction_hash
0,193.03,NaN,0xd66f016a40a8edf5232e06c7db3ef31346f556de50f2e58a372a6d252445e07e
1,100.79,NaN,0x33dfe6a14942db0be48a267feae724060554cbec675fa11877d742f9fdc219dc


,mq_amt_in_usd,nftfi_usd_value,nftfi_transaction_hash
20731,"12,820.53","12,824.07",0x862834c19fc5e3255d85ee631d45b38c210055fb8aa8ff51d71b3b64ee4d6cb2
20732,"12,818.90","12,851.30",0x0f94288dd02f42bb2a38bbacffb8c8d7afa8c7519f8db733d120d18a4863067f


/tmp/ipykernel_651408/550705424.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usd_diff['usd_diff_nftfi_minus_mq'] = usd_diff['nftfi_usd_value'] - usd_diff['mq_amt_in_usd']


,mq_amt_in_usd,nftfi_usd_value,nftfi_transaction_hash,usd_diff_nftfi_minus_mq
20731,"12,820.53","12,824.07",0x862834c19fc5e3255d85ee631d45b38c210055fb8aa8ff51d71b3b64ee4d6cb2,3.53
20732,"12,818.90","12,851.30",0x0f94288dd02f42bb2a38bbacffb8c8d7afa8c7519f8db733d120d18a4863067f,32.40


In [28]:
display(pd.DataFrame(usd_diff.min().values).T)
nftfi.loc[nftfi['nftfi_transaction_hash']=='0x7c7675a6613ddaded0b187314c2edd3ad3c5428a7a14e92830b74e03fcaf7210'].T
# display(pd.DataFrame(usd_diff.min().T['nftfi_transaction_hash'])

,0,1,2,3
0,0.01,0.00,0x0000a65c8f37e773beeb78d3bb8e4e4d1a58b301057265e9f06576b172ad9947,"-35,000.00"


,0
nftfi_loan_no,v1.loan.fixed-0
nftfi_id,0
nftfi_loan_contract,v1.loan.fixed
nftfi_status,repaid
nftfi_borrower,0xc35A5FEc6BE6957899E15559Be252Db882220b37
nftfi_lender,0x44DdF3e35Cd2D629d78674FF6BF5b953e2B069fe
nftfi_loan_principal_amount,0.03
nftfi_maximum_repayment_amount,0.00
nftfi_loan_duration_d,7
nftfi_apr,869.05


In [29]:
pd.DataFrame(usd_diff.usd_diff_nftfi_minus_mq.describe())

,usd_diff_nftfi_minus_mq
count,"19,849.00"
mean,-24.48
std,532.84
min,"-35,000.00"
25%,-4.64
50%,0.00
75%,4.30
max,"4,598.59"


## Check per collection borrow volume: all time, monthly, daily etc

In [30]:
start = '2023-03-01'
end = '2023-04-01'

april_mq = mq.loc[mq['mq_block_timestamp'] >= start].loc[mq['mq_block_timestamp'] < end]
display(april_mq.shape)
april_nftfi = nftfi.loc[nftfi['nftfi_date'] >= start].loc[nftfi['nftfi_date'] < end]
display(april_nftfi.shape)


april_full_outer = pd.merge(april_nftfi, april_mq, how='outer', left_on='nftfi_transaction_hash', right_on='mq_transaction_hash', indicator=True)

april_full_outer_without_nftfi_na = april_full_outer.dropna(subset=['nftfi_usd_value'])
april_full_outer_without_nftfi_na.head(2)

(1765, 21)

(3824, 28)

,nftfi_loan_no,nftfi_id,nftfi_loan_contract,nftfi_status,nftfi_borrower,nftfi_lender,nftfi_loan_principal_amount,nftfi_maximum_repayment_amount,nftfi_loan_duration_d,nftfi_apr,nftfi_loan_start_time,nftfi_date,nftfi_loan_erc20denomination,nftfi_block_num,nftfi_transaction_hash,nftfi_repaid,nftfi_liquidated,nftfi_loan_repaid_time,nftfi_repaid_amount_paid_to_lender,nftfi_repaid_transaction_hash,nftfi_loan_liquidation_date,nftfi_liquidated_transaction_hash,nftfi_loan_no_cleaned,nftfi_high,nftfi_eth_price,nftfi_usd_value,nftfi_loan_erc20denomination_name,nftfi_loan_type,mq_collection_name,mq_transaction_hash,mq_block_timestamp,mq_loan_id,mq_to_address,mq_from_address,mq_principal_amount,mq_repayment_amount,mq_erc20_address,mq_erc20_name,mq_due_date,mq_duration_in_days,mq_apr,mq_token_id,mq_collection_address,mq_protocol,mq_amt_in_usd,mq_roll_over,mq_block_number,mq_p2p_p2pool,mq_loan_id_cleaned,_merge
0,v2-1.loan.fixed-27220,27220,v2-1.loan.fixed,repaid,0xA7Ad4296D9f41434D1982A4C4cd98Df88121858F,0x5b20Dce335D131C6de0Cc11b481ACE2aaDe788DF,11.00,0.00,15,39.99,1970-01-01 00:00:01.677628991,2023-03-01 00:03:11+00:00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,16730087,0x825be250636b25fb40c60e94e16d46451663e0bf8fac52db2c02b2cf2d20ae81,True,True,2023-03-11 22:06:47+00:00,11171738150000000000,0xc3be1bac63c2be040b9439fe18924356ce9c45c280b28cf13da6c2590b9f29a2,NaN,NaN,27220,"1,605.47","1,604.89","17,653.79",weth,v2-1.loan.fixed,Land,0x825be250636b25fb40c60e94e16d46451663e0bf8fac52db2c02b2cf2d20ae81,2023-03-01 00:03:11.000000 UTC,"27,220.00",0xa7ad4296d9f41434d1982a4c4cd98df88121858f,0x5b20dce335d131c6de0cc11b481ace2aade788df,11.00,11.18,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-03-16 00:03:11.000000 UTC,15.00,0.40,511,0x2c88aa0956bc9813505d73575f653f69ada60923,nftfi,"17,659.74",False,"16,730,087.00",p2p,27220,both
1,v2-1.loan.fixed-27221,27221,v2-1.loan.fixed,repaid,0x7cb70995BCbB39D16F8cc3fD8E87580c5b3E37d2,0x9A650357D188DFAA743B95DDcfc92db731c2B7e3,0.20,0.00,30,49.00,1970-01-01 00:00:01.677630083,2023-03-01 00:21:23+00:00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,16730174,0x167054e7cd7991ed101f0a5d3f0aacb0a2bd5326679bdab5bc9a66d99863124b,True,True,2023-03-22 21:15:23+00:00,207652250000000000,0x02460dc9f3117beb2e60652cc75cda475f1181743997f156ad35697af5b7a686,NaN,NaN,27221,"1,609.17","1,608.41",321.68,weth,v2-1.loan.fixed,VeeFriends Series 2,0x167054e7cd7991ed101f0a5d3f0aacb0a2bd5326679bdab5bc9a66d99863124b,2023-03-01 00:21:23.000000 UTC,"27,221.00",0x7cb70995bcbb39d16f8cc3fd8e87580c5b3e37d2,0x9a650357d188dfaa743b95ddcfc92db731c2b7e3,0.20,0.21,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-03-31 00:21:23.000000 UTC,30.00,0.49,31142,0x9378368ba6b85c1fba5b131b530f5f5bedf21a18,nftfi,321.09,False,"16,730,174.00",p2p,27221,both


In [31]:
april_full_outer_without_nftfi_na = april_full_outer_without_nftfi_na.groupby('mq_collection_name').agg(sum)[['nftfi_usd_value', 'mq_amt_in_usd']]
april_full_outer_without_nftfi_na['nftfi_minus_mq'] = april_full_outer_without_nftfi_na['nftfi_usd_value'] - april_full_outer_without_nftfi_na['mq_amt_in_usd']
april_full_outer_without_nftfi_na.head(5)

,nftfi_usd_value,mq_amt_in_usd,nftfi_minus_mq
mq_collection_name,,,
0N1 Force,"11,404.01","13,634.18","-2,230.16"
0xmons,"8,280.35","8,288.58",-8.23
10KTF Gucci Grail,296.82,"7,282.03","-6,985.20"
Angry Ape Army,525.37,525.27,0.11
Art Blocks,"457,013.65","457,926.21",-912.57


In [32]:
april_full_outer_without_nftfi_na.describe()

,nftfi_usd_value,mq_amt_in_usd,nftfi_minus_mq
count,94.00,94.00,94.00
mean,"64,754.09","65,798.22","-1,044.13"
std,"159,909.29","161,346.06","3,060.74"
min,0.00,130.00,"-20,735.89"
25%,"1,337.40","1,460.90",-618.70
50%,"6,720.53","7,564.72",-15.19
75%,"52,957.36","54,345.89",-1.00
max,"993,306.92","1,014,042.82",195.26


In [33]:
pd.options.plotting.backend = "plotly"
april_full_outer_without_nftfi_na.plot(x=april_full_outer_without_nftfi_na.index, y=april_full_outer_without_nftfi_na.nftfi_minus_mq)

In [34]:
april_full_outer_without_nftfi_na.sort_values(by='nftfi_minus_mq')

,nftfi_usd_value,mq_amt_in_usd,nftfi_minus_mq
mq_collection_name,,,
MutantApeYachtClub,"993,306.92","1,014,042.82","-20,735.89"
Where My Vans Go,0.00,"14,500.00","-14,500.00"
Creepz by OVERLORD,"191,199.26","201,314.65","-10,115.39"
Terraforms,"31,869.47","40,566.62","-8,697.15"
Sewer Pass,"40,044.25","47,142.96","-7,098.71"
10KTF Gucci Grail,296.82,"7,282.03","-6,985.20"
Cryptoadz,"10,712.74","14,346.64","-3,633.90"
CryptoDickbutts S3,"198,929.51","202,026.81","-3,097.31"
Ethereum Name Service (ENS),"12,500.40","15,118.60","-2,618.20"


In [42]:
collection_of_interest = 'MutantApeYachtClub'
display(april_full_outer.loc[april_full_outer['mq_collection_name'] == collection_of_interest].describe())
display(april_full_outer.loc[april_full_outer['mq_collection_name'] == collection_of_interest])


,nftfi_id,nftfi_loan_principal_amount,nftfi_maximum_repayment_amount,nftfi_loan_duration_d,nftfi_apr,nftfi_block_num,nftfi_high,nftfi_eth_price,nftfi_usd_value,mq_loan_id,mq_principal_amount,mq_repayment_amount,mq_duration_in_days,mq_apr,mq_amt_in_usd,mq_block_number
count,57.00,57.00,57.00,57.00,57.00,57.00,57.00,57.00,57.00,57.00,57.00,57.00,57.00,57.00,57.00,57.00
mean,"29,212.93",803.88,0.00,29.21,25.34,"16,840,861.60","1,673.20","1,671.51","17,426.44","29,212.93","1,155.75","1,201.93",29.74,0.25,"17,790.22","16,840,861.60"
std,"1,059.50","3,402.30",0.00,22.48,13.23,"58,304.93",128.49,128.16,"4,127.73","1,059.50","4,249.40","4,447.91",22.85,0.13,"3,386.94","58,304.93"
min,"27,224.00",0.00,0.00,3.00,2.00,"16,730,461.00","1,383.37","1,383.05",0.00,"27,224.00",3.65,3.66,3.00,0.02,"6,426.87","16,730,461.00"
25%,"28,411.00",9.50,0.00,14.00,16.99,"16,796,093.00","1,565.32","1,564.41","15,745.01","28,411.00",9.60,9.75,14.00,0.17,"15,859.81","16,796,093.00"
50%,"29,344.00",10.50,0.00,30.00,23.00,"16,848,387.00","1,735.68","1,732.95","17,241.88","29,344.00",10.60,10.96,30.00,0.23,"17,543.36","16,848,387.00"
75%,"29,893.00",12.50,0.00,30.00,35.00,"16,877,207.00","1,776.46","1,772.59","19,562.12","29,893.00",13.00,13.10,30.00,0.35,"19,608.44","16,877,207.00"
max,"31,033.00","16,250.00",0.00,90.00,61.00,"16,949,976.00","1,833.08","1,829.04","23,971.26","31,033.00","20,057.00","22,068.20",90.00,0.61,"23,940.53","16,949,976.00"


,nftfi_loan_no,nftfi_id,nftfi_loan_contract,nftfi_status,nftfi_borrower,nftfi_lender,nftfi_loan_principal_amount,nftfi_maximum_repayment_amount,nftfi_loan_duration_d,nftfi_apr,nftfi_loan_start_time,nftfi_date,nftfi_loan_erc20denomination,nftfi_block_num,nftfi_transaction_hash,nftfi_repaid,nftfi_liquidated,nftfi_loan_repaid_time,nftfi_repaid_amount_paid_to_lender,nftfi_repaid_transaction_hash,nftfi_loan_liquidation_date,nftfi_liquidated_transaction_hash,nftfi_loan_no_cleaned,nftfi_high,nftfi_eth_price,nftfi_usd_value,nftfi_loan_erc20denomination_name,nftfi_loan_type,mq_collection_name,mq_transaction_hash,mq_block_timestamp,mq_loan_id,mq_to_address,mq_from_address,mq_principal_amount,mq_repayment_amount,mq_erc20_address,mq_erc20_name,mq_due_date,mq_duration_in_days,mq_apr,mq_token_id,mq_collection_address,mq_protocol,mq_amt_in_usd,mq_roll_over,mq_block_number,mq_p2p_p2pool,mq_loan_id_cleaned,_merge
4,v2.loan.fixed.collection-27224,27224,v2.loan.fixed.collection,NaN,0xE5CA890A0eF2F128EB3267e4711c6bf3306Ec024,0x26fCbD3AFEbbE28D0A8684F790C48368D21665b5,10.00,0.00,60,28.00,1970-01-01 00:00:01.677633551,2023-03-01 01:19:11+00:00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,16730461,0x8411955e02bbce1a27c841e8a0bd5f442db07bb9dcdbba7b79df7eddd8c390c5,True,True,NaT,NaN,NaN,NaN,NaN,27224,"1,607.44","1,606.64","16,066.40",weth,v2.loan.fixed.collection,MutantApeYachtClub,0x8411955e02bbce1a27c841e8a0bd5f442db07bb9dcdbba7b79df7eddd8c390c5,2023-03-01 01:19:11.000000 UTC,"27,224.00",0xe5ca890a0ef2f128eb3267e4711c6bf3306ec024,0x26fcbd3afebbe28d0a8684f790c48368d21665b5,10.00,10.46,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-30 01:19:11.000000 UTC,60.00,0.28,22584,0x60e4d786628fea6478f785a6d7e704777c86a7c6,nftfi,"16,017.90",False,"16,730,461.00",p2p,27224,both
31,v2-1.loan.fixed-27251,27251,v2-1.loan.fixed,repaid,0x371267a1863080c5F39229183350e0D87d39743E,0x8d4169929C0c73A93684B23dC8b7A72D48F2FC58,14.50,0.00,21,35.00,1970-01-01 00:00:01.677659687,2023-03-01 08:34:47+00:00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,16732617,0x9e003c9fbccbb6c4ab24964284bedfe0a8d73395996754bf2e919f126c671491,True,True,2023-03-22 02:12:47+00:00,14777386700000000000,0x2661377f4e76a1fec46d517609e5d3726ceed7f9703d0c321259deb6cefb6978,NaN,NaN,27251,"1,654.20","1,653.19","23,971.26",weth,v2-1.loan.fixed,MutantApeYachtClub,0x9e003c9fbccbb6c4ab24964284bedfe0a8d73395996754bf2e919f126c671491,2023-03-01 08:34:47.000000 UTC,"27,251.00",0x371267a1863080c5f39229183350e0d87d39743e,0x8d4169929c0c73a93684b23dc8b7a72d48f2fc58,14.50,14.79,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-03-22 08:34:47.000000 UTC,21.00,0.35,15013,0x60e4d786628fea6478f785a6d7e704777c86a7c6,nftfi,"23,940.53",False,"16,732,617.00",p2p,27251,both
300,v2.loan.fixed.collection-27520,27520,v2.loan.fixed.collection,repaid,0x0e3BcD1efd366092248aC733Af93F57fDABA5A12,0xD4E26683635bf3dc9EAD5F31B935c33cC1Ce1838,12.50,0.00,30,15.00,1970-01-01 00:00:01.677827483,2023-03-03 07:11:23+00:00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,16746415,0x77c4be61bf3df7946d70a2138a9b6bf215634feefad8711487fe295ba9e6ea3b,True,True,2023-03-04 05:36:11+00:00,12646404500000000000,0xf9ac72a4d83004271754cd0e6631817cadb0841d6b830e9e145780474903046c,NaN,NaN,27520,"1,565.32","1,564.97","19,562.12",weth,v2.loan.fixed.collection,MutantApeYachtClub,0x77c4be61bf3df7946d70a2138a9b6bf215634feefad8711487fe295ba9e6ea3b,2023-03-03 07:11:23.000000 UTC,"27,520.00",0x0e3bcd1efd366092248ac733af93f57fdaba5a12,0xd4e26683635bf3dc9ead5f31b935c33cc1ce1838,12.50,12.65,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,wETH,2023-04-02 07:11:23.000000 UTC,30.00,0.15,16625,0x60e4d786628fea6478f785a6d7e704777c86a7c6,nftfi,"19,570.11",False,"16,746,415.00",p2p,27520,both
313,v2-1.loan.fixed-27533,27533,v2-1.loan.fixed,repaid,0x371267a1863080c5F39229183350e0D87d39743E,0xD4E26683635bf3dc9EAD5F31B935c33cC1Ce1838,15.00,0.00,30,20.00,1970-01-01 00:00:01.677844727,2023-03-03 11:58:47+00:00,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,16747837,0x940af410f0f43809637520a

In [43]:
display(april_full_outer.loc[april_full_outer['mq_collection_name'] == collection_of_interest].loc[april_full_outer['_merge'] != 'both']['_merge'].unique())
april_full_outer.loc[april_full_outer['mq_collection_name'] == collection_of_interest].loc[april_full_outer['_merge'] != 'both']

[], Categories (3, object): ['left_only', 'right_only', 'both']

,nftfi_loan_no,nftfi_id,nftfi_loan_contract,nftfi_status,nftfi_borrower,nftfi_lender,nftfi_loan_principal_amount,nftfi_maximum_repayment_amount,nftfi_loan_duration_d,nftfi_apr,nftfi_loan_start_time,nftfi_date,nftfi_loan_erc20denomination,nftfi_block_num,nftfi_transaction_hash,nftfi_repaid,nftfi_liquidated,nftfi_loan_repaid_time,nftfi_repaid_amount_paid_to_lender,nftfi_repaid_transaction_hash,nftfi_loan_liquidation_date,nftfi_liquidated_transaction_hash,nftfi_loan_no_cleaned,nftfi_high,nftfi_eth_price,nftfi_usd_value,nftfi_loan_erc20denomination_name,nftfi_loan_type,mq_collection_name,mq_transaction_hash,mq_block_timestamp,mq_loan_id,mq_to_address,mq_from_address,mq_principal_amount,mq_repayment_amount,mq_erc20_address,mq_erc20_name,mq_due_date,mq_duration_in_days,mq_apr,mq_token_id,mq_collection_address,mq_protocol,mq_amt_in_usd,mq_roll_over,mq_block_number,mq_p2p_p2pool,mq_loan_id_cleaned,_merge


In [57]:
# TODO: compute the implied ETH price 
april_full_outer_for_collection = april_full_outer.loc[april_full_outer['mq_collection_name'] == collection_of_interest]
april_full_outer_for_collection['nftfi_minus_mq'] = april_full_outer_for_collection['nftfi_usd_value'] - april_full_outer_for_collection['mq_amt_in_usd']
pd.DataFrame(april_full_outer_for_collection['nftfi_minus_mq'].describe()).T

/tmp/ipykernel_651408/3243541687.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,count,mean,std,min,25%,50%,75%,max
nftfi_minus_mq,57.00,-363.79,"2,656.93","-20,057.00",-58.07,-4.86,23.02,297.54


In [67]:
april_full_outer_for_collection.dtypes
# TODO: loc for wETH address
april_full_outer_for_collection['mq_implied_eth_price'] = april_full_outer_for_collection['mq_amt_in_usd'] / april_full_outer_for_collection['mq_principal_amount']
april_full_outer_for_collection['nftfi_mq_eth_price_diff'] = april_full_outer_for_collection['nftfi_eth_price'] - april_full_outer_for_collection['mq_implied_eth_price']
april_full_outer_for_collection[['mq_transaction_hash','nftfi_minus_mq', 'nftfi_eth_price','mq_implied_eth_price', 'nftfi_mq_eth_price_diff']].sort_values('nftfi_minus_mq')

/tmp/ipykernel_651408/3581407080.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_651408/3581407080.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,mq_transaction_hash,nftfi_minus_mq,nftfi_eth_price,mq_implied_eth_price,nftfi_mq_eth_price_diff
3816,0xeebf7ab85ff087d5b008ff79a56496280bef7b44d26923ba0f72491089131671,"-20,057.00","1,824.92",1.00,"1,823.92"
2052,0xf406c61f52dc153ad159adb8d6c89c4b9a6accb475db36e56d7d6698bb3452a1,-296.42,"1,656.05","1,684.51",-28.46
2053,0xa3d8fa2d2e4215d01b3e45669d84977db1b244e9602a7ca430f1f40ea989a796,-227.54,"1,656.07","1,684.51",-28.44
839,0x3f00c5cb30894fd3d130be5e6f076c33da6729f074f6228a0c75f057471135c6,-174.74,"1,549.66","1,562.60",-12.94
3398,0xabcf54d7d12eaf7a85c24646702d31ca7d55355cd359e53ac1bb87868126a219,-166.33,"1,737.93","1,755.63",-17.70
1191,0xdf1ab008dd740196ae850b9a6233e57677181450866670a7701bd7724f325e08,-154.64,"1,407.22","1,420.11",-12.89
1521,0x0b22c21aa0dddc8a73f0c831e593dd66b12f03be4bb349b5c4bb1d8814baa7aa,-148.06,"1,464.09","1,477.31",-13.22
2014,0x460313c27cfa927391b464d4d09e951f17753be51d05323eed5ba63a2e41aed3,-114.79,"1,657.37","1,669.45",-12.08
3183,0xbe4d566975bfc45985c1f07898e6ffa4daf82161dfc1598fab2918941113103e,-108.66,"1,744.20","1,754.61",-10.41
2476,0xd7451478f0501e74072f29a92ce954f78c39de11350176e83ca8157cd16ed7f1,-103.31,"1,802.41","1,810.36",-7.95


In [68]:
april_full_outer_for_collection.loc[april_full_outer_for_collection['mq_transaction_hash'] == '0xeebf7ab85ff087d5b008ff79a56496280bef7b44d26923ba0f72491089131671']

,nftfi_loan_no,nftfi_id,nftfi_loan_contract,nftfi_status,nftfi_borrower,nftfi_lender,nftfi_loan_principal_amount,nftfi_maximum_repayment_amount,nftfi_loan_duration_d,nftfi_apr,nftfi_loan_start_time,nftfi_date,nftfi_loan_erc20denomination,nftfi_block_num,nftfi_transaction_hash,nftfi_repaid,nftfi_liquidated,nftfi_loan_repaid_time,nftfi_repaid_amount_paid_to_lender,nftfi_repaid_transaction_hash,nftfi_loan_liquidation_date,nftfi_liquidated_transaction_hash,nftfi_loan_no_cleaned,nftfi_high,nftfi_eth_price,nftfi_usd_value,nftfi_loan_erc20denomination_name,nftfi_loan_type,mq_collection_name,mq_transaction_hash,mq_block_timestamp,mq_loan_id,mq_to_address,mq_from_address,mq_principal_amount,mq_repayment_amount,mq_erc20_address,mq_erc20_name,mq_due_date,mq_duration_in_days,mq_apr,mq_token_id,mq_collection_address,mq_protocol,mq_amt_in_usd,mq_roll_over,mq_block_number,mq_p2p_p2pool,mq_loan_id_cleaned,_merge,nftfi_minus_mq,mq_implied_eth_price,nftfi_mq_eth_price_diff
3816,v2-1.loan.fixed-31033,31033,v2-1.loan.fixed,NaN,0xbf37f27A4ac7BD29290Ed2c3aA68e6ab11Fd4d6e,0xd5f6ed8D6743c1956190Da6dAE115E5fB7141ed7,0.00,0.00,30,61.00,1970-01-01 00:00:01.680299603,2023-03-31 21:53:23+00:00,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,16949976,0xeebf7ab85ff087d5b008ff79a56496280bef7b44d26923ba0f72491089131671,True,True,NaT,NaN,NaN,NaN,NaN,31033,"1,827.49","1,824.92",0.00,usdc,v2-1.loan.fixed,MutantApeYachtClub,0xeebf7ab85ff087d5b008ff79a56496280bef7b44d26923ba0f72491089131671,2023-03-31 21:53:23.000000 UTC,"31,033.00",0xbf37f27a4ac7bd29290ed2c3aa68e6ab11fd4d6e,0xd5f6ed8d6743c1956190da6dae115e5fb7141ed7,"20,057.00","22,068.20",0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDc,2023-05-30 21:53:23.000000 UTC,60.00,0.61,1542,0x60e4d786628fea6478f785a6d7e704777c86a7c6,nftfi,"20,057.00",False,"16,949,976.00",p2p,31033,both,"-20,057.00",1.00,"1,823.92"


## Compute April subset for each table

In [ ]:
april_mq = mq.loc[mq['block_timestamp'] >= '2023-04-01'].loc[mq['block_timestamp'] < '2023-05-01']
april_mq.shape

In [ ]:
april_nftfi = nftfi.loc[nftfi['date'] >= '2023-04-01'].loc[nftfi['date'] < '2023-05-01']
april_nftfi.shape

#### Check for duplicates

In [ ]:
april_nftfi.loc[april_nftfi.loan_no_cleaned.duplicated()== True]

In [ ]:
display(april_mq.loc[april_mq.loan_id_cleaned.duplicated()== True].head(5))
display(april_mq.loc[april_mq.loan_id_cleaned.duplicated()== True].shape[0])

In [ ]:
april_nftfi.loc[april_nftfi.loan_no_cleaned.duplicated()== True]

In [ ]:
display(april_mq.head(3).sort_values('loan_id_cleaned', ascending=False))
display(april_nftfi.head(3).sort_values('loan_no_cleaned', ascending=False))

In [ ]:
# Determine the set of unique IDs present in MQ and not in NFTfi
len(set(list(april_mq.loan_id_cleaned)) - set(list(april_nftfi.loan_no_cleaned)))

In [ ]:
# Determine the set of unique IDs present in NFTfi and not in MQ's
len(set(list(april_nftfi.loan_no_cleaned)) - set(list(april_mq.loan_id_cleaned)))

In [ ]:
april_mq.shape, april_nftfi.shape

In [ ]:
april_nftfi.shape[0] - april_mq.shape[0]

## Compute which Loans are available in NFTfi's and not in MQ's, and the converse, by matching on transaction hashes

In [ ]:
# how do i get all rows in df1 that are not in df2
# and all rows in df2 that are not in df1?

# Perform a full outer join on df1 and df2
full_outer = pd.merge(april_nftfi, april_mq, how='outer', on='transaction_hash', indicator=True)

# Filter out the rows that are unique to df1
unique_to_april_nftfi = full_outer[full_outer['_merge'] == 'left_only']

# Filter out the rows that are unique to df2
unique_to_april_mq = full_outer[full_outer['_merge'] == 'right_only']


In [ ]:
print(f"number of loans unique to NFTfi relative to MQ in April 2023: {unique_to_april_nftfi.shape[0]}, i.e. {100 * round(unique_to_april_nftfi.shape[0] / april_nftfi.shape[0], 2)}%")
print(f"while in April 2023 NFTfi had {april_nftfi.shape[0]} loans while mq had {april_mq.shape[0]}, i.e. for April NFTfi has {april_nftfi.shape[0] - april_mq.shape[0]} more loans")
display(unique_to_april_nftfi.shape[0])
display(unique_to_april_nftfi.head(2))

In [ ]:
print(f"number of loans unique to MQ relative to NFTfi in April 2023: {unique_to_april_mq.shape[0]}, i.e. {100 * round(unique_to_april_mq.shape[0] / april_mq.shape[0], 2)}%")
print(f"while in April 2023 NFTfi had {april_nftfi.shape[0]} loans while mq had {april_mq.shape[0]}, i.e. for April NFTfi has {april_nftfi.shape[0] - april_mq.shape[0]} more loans")

display(unique_to_april_mq.shape[0])
display(unique_to_april_mq.head(2))

In [ ]:
pd.set_option('display.float_format', '{:,.0f}'.format)
pd.DataFrame(unique_to_april_nftfi.groupby('loan_contract').agg(sum)['usd_value'])

In [ ]:
# MQ loans not in NFTfi
# unique_to_april_mq = april_mq[april_mq['loan_id_cleaned'].isin(mq_loans_not_in_nftfi)].reset_index(drop=True)
unique_to_april_mq.head(3)

In [ ]:
# We need to explore this table
unique_to_april_mq.erc20_name.unique()

## NFTfi loans not in MQ

In [ ]:
unique_to_april_nftfi

In [ ]:
# Explore the resulting set of unique_to_april_nftfi. Is there any pattern?
unique_to_april_nftfi.describe()

### Are discrepancies due to loan type which is not accounted for?

In [ ]:
# How about loan numbers, how do they vary from a naming standpoint?
# x[0]+x[1] if len(x) >2 else x[0] is such that it returns "v2-1.loan.fixed" from "v2-1.loan.fixed-32381", 
#    else returns "v2.loan.fixed.collection" from "v2.loan.fixed.collection-34479"

unique_to_april_nftfi.loan_no.astype(str).str.split('-').apply(lambda x: x[0]+'-'+x[1] if len(x) >2 else x[0]).unique()
# it seems from the above result that [v2-1.loan.fixed', 'v2.loan.fixed.collection'] are the two loan types which are not accounted for

In [ ]:
# Is the above true as well for the whole table set?
nftfi_loans_not_in_mq = list(
        set(list(nftfi.loan_no_cleaned)) - set(list(mq.loan_id_cleaned))
    )
print(len(nftfi_loans_not_in_mq))
# How about loan numbers, how do they vary from a naming standpoint?
# x[0]+x[1] if len(x) >2 else x[0] is such that it returns "v2-1.loan.fixed" from "v2-1.loan.fixed-32381", 
#    else returns "v2.loan.fixed.collection" from "v2.loan.fixed.collection-34479"

unique_to_april_nftfi.loan_no.astype(str).str.split('-').apply(lambda x: x[0]+'-'+x[1] if len(x) >2 else x[0]).unique()
# it seems from the above result that [v2-1.loan.fixed', 'v2.loan.fixed.collection'] are the two loan types which are not accounted for

### Is it due to missed erc20 token denomination?

In [ ]:
# Check if it can be due to missing token
usdc_address = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'  # https://etherscan.io/token/0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
usdt_address = '0xdAC17F958D2ee523a2206206994597C13D831ec7'  # https://etherscan.io/token/0xdac17f958d2ee523a2206206994597c13d831ec7
dai_address = '0x6B175474E89094C44Da98b954EedeAC495271d0F'  # https://etherscan.io/token/0x6b175474e89094c44da98b954eedeac495271d0f
weth_address = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'

stablecoin_addresses = [usdc_address, usdt_address, dai_address, weth_address]

stablecoin_names = ['usdc', 'usdt', 'dai', 'weth']
for stablecoin_address, stablecoin_name in zip(stablecoin_addresses, stablecoin_names):
    stablecoin_address = stablecoin_address.lower()
    unique_to_april_nftfi.loc[unique_to_april_nftfi['loan_erc20denomination'] == stablecoin_address, 'loan_erc20denomination_name'] = stablecoin_name

In [ ]:
unique_to_april_nftfi.loan_erc20denomination_name.unique()
# It does not seem to be linked to missing token.

### Let's compute loan volumes per token denomination to see if they match

In [ ]:
display(april_nftfi.head(2))
display(april_mq.head(2))

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.float_format', '{:,.2f}'.format)
display(april_nftfi.groupby('loan_erc20denomination_name').agg('sum'))

display(april_mq.groupby('erc20_name').agg('sum'))


In [ ]:

print("NFTfi:")
display(pd.DataFrame(april_nftfi.groupby('loan_erc20denomination_name').agg('sum')['loan_principal_amount']))
print("\n\nMQ:")
display(pd.DataFrame(april_mq.groupby('erc20_name').agg('sum')['principal_amount']))

#### We see from the above results that NFTfi and MQ respective loan principal amounts grouped by the loan erc20 denomination do not match at all. How about the total dollar numbers?

In [ ]:
print("NFTfi:")
total_usd_nftfi_volume_per_ccy = pd.DataFrame(april_nftfi.groupby('loan_erc20denomination_name').agg('sum')['usd_value'])
display(total_usd_nftfi_volume_per_ccy)
print("\n\nMQ:")
total_usd_mq_volume_per_ccy = pd.DataFrame(april_mq.groupby('erc20_name').agg('sum')['amt_in_usd'])
display(total_usd_mq_volume_per_ccy)

#### Total numbers are very off. Can that be explained with the set of loans which are not in each dataset?

In [ ]:
print("Total dollar value of loan principal for NFTfi loans which are not in MQ, for April and NFTfi:")
display(pd.DataFrame(unique_to_april_nftfi.groupby('loan_erc20denomination_name').agg('sum')['usd_value']))
print("\n\nTotal dollar value of loan principal for MQ loans which are not inNFtfi, for April and MQ:")
display(pd.DataFrame(unique_to_april_mq.groupby('erc20_name').agg('sum')['amt_in_usd']))

In [ ]:
usd_values_of_nftfi_not_in_mq = pd.DataFrame(unique_to_april_nftfi.groupby('loan_erc20denomination_name').agg('sum')['usd_value'])
usd_values_of_mq_not_in_nftfi = pd.DataFrame(unique_to_april_mq.groupby('erc20_name').agg('sum')['amt_in_usd'])

In [ ]:
weth_usd_amount_nftfi_not_in_mq = usd_values_of_nftfi_not_in_mq.loc['weth'].values[0]
weth_usd_amount_mq_not_in_nftfi = usd_values_of_mq_not_in_nftfi.loc['wETH'].values[0]

weth_usd_amount_total_nftfi = total_usd_nftfi_volume_per_ccy.loc['weth'].values[0]
weth_usd_amount_total_mq = total_usd_mq_volume_per_ccy.loc['wETH'].values[0]

print(weth_usd_amount_total_nftfi, weth_usd_amount_nftfi_not_in_mq, weth_usd_amount_mq_not_in_nftfi, weth_usd_amount_total_mq)

# total NFTfi missing from MQ volume, minus total MQ volume missing from NFTfi

weth_usd_amount_nftfi_not_in_mq - weth_usd_amount_mq_not_in_nftfi - weth_usd_amount_total_nftfi  + weth_usd_amount_total_mq

#### How about dollar amounts PER loan type PER ccy?

In [ ]:
usd_values_of_nftfi_not_in_mq = pd.DataFrame(unique_to_april_nftfi.groupby(['loan_erc20denomination_name','loan_type']).agg('sum')['usd_value'])
usd_values_of_nftfi_not_in_mq

In [ ]:
print("NFTfi:")
display(total_usd_nftfi_volume_per_ccy)
print("\n\nMQ:")
display(total_usd_mq_volume_per_ccy)

## For the loan IDs which do match... are they the same loans?

In [ ]:
import copy
nftfi_with_col_prefix = copy.deepcopy(nftfi)
mq_with_col_prefix = copy.deepcopy(mq)
nftfi_with_col_prefix = nftfi_with_col_prefix.rename(columns={col: 'nftfi_'+ col for col in nftfi.columns})
mq_with_col_prefix = mq_with_col_prefix.rename(columns={col: 'mq_'+ col for col in mq.columns})

matching_mq_with_nftfi = pd.merge(left=nftfi_with_col_prefix, right=mq_with_col_prefix, left_on='nftfi_loan_no_cleaned', right_on='mq_loan_id_cleaned', how='inner', suffixes=('nftfi', 'mq'))
matching_mq_with_nftfi

In [ ]:
subset_of_matching_mq_with_nftfi = matching_mq_with_nftfi[['nftfi_date', 'mq_block_timestamp', 'nftfi_usd_value', 'mq_amt_in_usd', 'nftfi_loan_no_cleaned', 'nftfi_loan_type', 'mq_loan_id_cleaned']]
subset_of_matching_mq_with_nftfi

In [ ]:
pd.options.plotting.backend = "plotly"
subset_of_matching_mq_with_nftfi['dollar_diff'] = subset_of_matching_mq_with_nftfi['nftfi_usd_value'] - subset_of_matching_mq_with_nftfi['mq_amt_in_usd']

subset_of_matching_mq_with_nftfi.nftfi_date = pd.to_datetime(subset_of_matching_mq_with_nftfi.nftfi_date)
subset_of_matching_mq_with_nftfi.mq_block_timestamp = pd.to_datetime(subset_of_matching_mq_with_nftfi.mq_block_timestamp)

subset_of_matching_mq_with_nftfi.plot.scatter(x=subset_of_matching_mq_with_nftfi.nftfi_date, y=subset_of_matching_mq_with_nftfi.dollar_diff)

In [ ]:
subset_of_matching_mq_with_nftfi['date_diff'] = subset_of_matching_mq_with_nftfi['nftfi_date'] - subset_of_matching_mq_with_nftfi['mq_block_timestamp']
subset_of_matching_mq_with_nftfi.plot.scatter(x=subset_of_matching_mq_with_nftfi.nftfi_date, y=subset_of_matching_mq_with_nftfi.date_diff)

In [ ]:
import numpy as np
subset_of_matching_mq_with_nftfi['date_diff'] = subset_of_matching_mq_with_nftfi['nftfi_date'] - subset_of_matching_mq_with_nftfi['mq_block_timestamp']
non_zero_time_delta = subset_of_matching_mq_with_nftfi.loc[subset_of_matching_mq_with_nftfi['date_diff'] > np.timedelta64(0)]
display(non_zero_time_delta)
display(non_zero_time_delta.nftfi_loan_type.unique())

print(f"min, max nftfi [{non_zero_time_delta.nftfi_date.min(), non_zero_time_delta.nftfi_date.max()}]")
print(f"min, max mq [{non_zero_time_delta.mq_block_timestamp.min(), non_zero_time_delta.mq_block_timestamp.max()}]")

print(f"\n\nTotal dollar diff for non zero time deltas (IDs not matching) [{non_zero_time_delta.dollar_diff.sum()}]")
print(f"Total dollar diff for both datasets [{subset_of_matching_mq_with_nftfi.dollar_diff.sum()}]")

In [ ]:
nftfi.date = pd.to_datetime(nftfi.date)
time_delta = pd.Timedelta(minutes=20)
datemax_minus_time_delta = non_zero_time_delta.nftfi_date.max() - time_delta
datemax_plus_time_delta = non_zero_time_delta.nftfi_date.max() + time_delta

print(non_zero_time_delta.nftfi_date.max())
nftfi.loc[nftfi.date < datemax_plus_time_delta].loc[datemax_minus_time_delta < nftfi.date].sort_values('date')

In [ ]:
non_zero_time_delta.loc[non_zero_time_delta['dollar_diff'] == 0]

### Are missing loans from MQ relative to NFTfi, bundles?

In [ ]:
display(unique_to_april_nftfi.collection_name.unique())
len(unique_to_april_nftfi.collection_name.unique())

In [ ]:
display(unique_to_april_mq.collection_address.unique()[:3])
len(unique_to_april_mq.collection_address.unique())

## How are liquidations accounted for?

In [ ]:
# TODO:
# spreadsheet of different transactions that are missing, or are more of them on MQ dataset. we need clear doc discussing which tsx are missing, which are added,
# what are the volume differences coming from.

# try and check that the eth value of those loans per transaction is the same.
# check if volumes per loanerc20 collateral are matching

## Sandbox

In [ ]:
try: 
    volume = pd.read_csv('analysis/nftfi_cluster_data/3month_200apr_with_borrow_volume.csv')
    base_data = pd.read_csv('analysis/nftfi_cluster_data/20230522_CADLabs_NFTfi NFT collection APR ranges_vDraft - past 3M; loan count 20; APRs 200; 5 clusters.csv')
except FileNotFoundError:
    volume = (
        pd.read_csv('nftfi_cluster_data/3month_200apr_with_borrow_volume.csv')
        .drop(columns=['mean', 'median', 'std', 'count'])
    )
    base_data = (
        pd.read_csv('nftfi_cluster_data/20230522_CADLabs_NFTfi NFT collection APR ranges_vDraft - past 3M; loan count 20; APRs 200; 5 clusters.csv')
        .drop(columns=['Unnamed: 0', 'Unnamed: 7', 'Unnamed: 8'])
    )

In [ ]:
display(volume.head(2))
display(base_data.head(2))

In [ ]:
nftfi = pd.merge(left=base_data, right=volume, on='collection_name')
nftfi 
# list(nftfi['borrow_volume'].values)